In [89]:
import torch
import numpy as np

In [90]:
w = torch.tensor(2.8, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

x = torch.tensor([1.0, 2.0])

In [91]:
y_pred = w * x + b
print(f"y_pred: {y_pred}")

y_pred: tensor([3.8000, 6.6000], grad_fn=<AddBackward0>)


In [92]:
y_true = torch.tensor([10.0, 10.0])
loss = ((y_true - y_pred) ** 2).mean()
print(f"loss: {loss.item()}")

loss: 25.0


In [93]:
print(f"w before backward: w.grad = {w.grad}")

w before backward: w.grad = None


In [94]:
loss.backward()


In [95]:
print(f"apres backward: w.grad = {w.grad}")
print(f"apres backward: b.grad = {b.grad}")

apres backward: w.grad = -13.0
apres backward: b.grad = -9.600000381469727


In [96]:
learning_rate = 0.01
with torch.no_grad():
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad


In [97]:
print(f"nouveau poids: w = {w}, b = {b}")

nouveau poids: w = 2.929999828338623, b = 1.0959999561309814


In [98]:
import torch.nn as nn
perception = nn.Linear(1, 1)

print(f"poids: {perception.weight}")
print(f"biais: {perception.bias}")

poids: Parameter containing:
tensor([[0.7671]], requires_grad=True)
biais: Parameter containing:
tensor([-0.8223], requires_grad=True)


In [99]:
# Créer un tensor de la forme (batch_size, in_features)
# perception attend des entrées de forme (N, 1) car nn.Linear(1,1)
x = torch.tensor([[3.0]])  # batch de 2 échantillons, 1 feature chacun
print(f"input: {x}")
y_prod = perception(x)
print(y_prod)

input: tensor([[3.]])
tensor([[1.4790]], grad_fn=<AddmmBackward0>)


In [100]:
y_true = torch.tensor([[10.0]])
criterium = nn.MSELoss()
loss = criterium(y_prod, y_true)
print(f"loss: {loss.item()}")

loss: 72.60714721679688


In [101]:
loss.backward()

In [102]:
print(f"gradient du poids: {perception.weight.grad}")
print(f"gradient du biais: {perception.bias.grad}")

gradient du poids: tensor([[-51.1259]])
gradient du biais: tensor([-17.0420])


In [103]:
import torch.optim as optim
optimizer = optim.SGD(perception.parameters(), lr=0.01)


In [104]:
optimizer.step()


In [105]:
print(f"nouveau poids : {perception.weight} {perception.bias}")

nouveau poids : Parameter containing:
tensor([[1.2784]], requires_grad=True) Parameter containing:
tensor([-0.6519], requires_grad=True)


In [106]:
optimizer.zero_grad()

In [107]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [108]:
model = SimpleNet()

In [109]:
model

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [110]:
# Entrée : un batch d'un seul échantillon avec 2 features (SimpleNet attend 2 features)
x = torch.tensor([[1.0, 2.0]])  # forme (1, 2)
y = torch.tensor([[5.0]])  # forme (1, 1)
y_pred = model(x)
print(y_pred)

tensor([[0.4069]], grad_fn=<SigmoidBackward0>)


In [111]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose
import matplotlib.pyplot as plt


In [112]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [113]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [114]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [115]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.fc(x)
        return logits


In [116]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [117]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)

In [118]:
logits

tensor([[-0.0733,  0.0767,  0.1760,  0.1174,  0.0067, -0.0466,  0.0403, -0.0212,
         -0.1393, -0.0312]], grad_fn=<AddmmBackward0>)

In [119]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0916, 0.1064, 0.1175, 0.1108, 0.0992, 0.0941, 0.1026, 0.0965, 0.0857,
         0.0955]], grad_fn=<SoftmaxBackward0>)

In [120]:
y_pred = pred_probab.argmax(1)
y_pred

tensor([2])

In [121]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [122]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader):
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        # Calcul des prédictions et de la perte
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [123]:
def test(dataloader, model, loss_fn):
    # Si on passe un dataset directement, on le transforme en DataLoader
    if isinstance(dataloader, torch.utils.data.Dataset):
        dataloader = DataLoader(dataloader, batch_size=64)

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0.0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = 100.0 * correct / size
    print(f"Test set: Accuracy: {accuracy:0.1f}%, Avg loss: {test_loss:>8f}")


In [124]:
epoche = 10
for t in range(epoche):
    print(f"epoche {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)
print("Done!")

epoche 1
-------------------------------
loss: 2.311970  [   64/60000]
loss: 2.309098  [ 6464/60000]
loss: 2.299916  [12864/60000]
loss: 2.296229  [19264/60000]
loss: 2.291322  [25664/60000]
loss: 2.283035  [32064/60000]
loss: 2.276481  [38464/60000]
loss: 2.273119  [44864/60000]
loss: 2.260841  [51264/60000]
loss: 2.265256  [57664/60000]
Test set: Accuracy: 25.6%, Avg loss: 2.259522
epoche 2
-------------------------------
loss: 2.259753  [   64/60000]
loss: 2.256853  [ 6464/60000]
loss: 2.240374  [12864/60000]
loss: 2.243146  [19264/60000]
loss: 2.238536  [25664/60000]
loss: 2.242663  [32064/60000]
loss: 2.224883  [38464/60000]
loss: 2.211133  [44864/60000]
loss: 2.212622  [51264/60000]
loss: 2.210699  [57664/60000]
Test set: Accuracy: 52.9%, Avg loss: 2.197346
epoche 3
-------------------------------
loss: 2.192114  [   64/60000]
loss: 2.198906  [ 6464/60000]
loss: 2.170607  [12864/60000]
loss: 2.192318  [19264/60000]
loss: 2.155690  [25664/60000]
loss: 2.163539  [32064/60000]
loss:

In [125]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms


class ImageClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [126]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) 
])

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [127]:
model = ImageClassifierModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [128]:
print("Starting Model Training...")
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print("Finished Training.")

Starting Model Training...
[Epoch 1, Batch   100] loss: 1.604
[Epoch 1, Batch   200] loss: 0.338
[Epoch 1, Batch   300] loss: 0.197
[Epoch 1, Batch   400] loss: 0.161
[Epoch 1, Batch   500] loss: 0.124
[Epoch 1, Batch   600] loss: 0.106
[Epoch 1, Batch   700] loss: 0.104
[Epoch 1, Batch   800] loss: 0.096
[Epoch 1, Batch   900] loss: 0.090
[Epoch 2, Batch   100] loss: 0.076
[Epoch 2, Batch   200] loss: 0.070
[Epoch 2, Batch   300] loss: 0.069
[Epoch 2, Batch   400] loss: 0.073
[Epoch 2, Batch   500] loss: 0.066
[Epoch 2, Batch   600] loss: 0.063
[Epoch 2, Batch   700] loss: 0.059
[Epoch 2, Batch   800] loss: 0.060
[Epoch 2, Batch   900] loss: 0.058
[Epoch 3, Batch   100] loss: 0.048
[Epoch 3, Batch   200] loss: 0.049
[Epoch 3, Batch   300] loss: 0.051
[Epoch 3, Batch   400] loss: 0.047
[Epoch 3, Batch   500] loss: 0.040
[Epoch 3, Batch   600] loss: 0.047
[Epoch 3, Batch   700] loss: 0.039
[Epoch 3, Batch   800] loss: 0.052
[Epoch 3, Batch   900] loss: 0.044
[Epoch 4, Batch   100] loss:

In [129]:
print("Exporting TRAINED model to ONNX...")
model.eval()

example_inputs = torch.randn(1, 1, 28, 28) 

torch.onnx.export(
    model, 
    example_inputs, 
    f="image_classifier_model.onnx",
    input_names=['input'], 
    output_names=['output'],
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    dynamo=False
)

print("\n--- SUCCESS ---")

Exporting TRAINED model to ONNX...

--- SUCCESS ---


C:\Users\Laerthe\AppData\Local\Temp\ipykernel_23636\4041265909.py:6: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
